<a href="https://colab.research.google.com/github/jasonola/spacy/blob/master/Spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> Notes : 


*   Les instructions pour commenter votre code sont sur le moodle du cours, dans la section "Commenter du code".
*   Pour installer des librairies dans le colab, utilisez la commande 


```
!pip install
```

a) Téléchargez la liste de romans en .txt présente ici https://drive.switch.ch/index.php/s/b5j3otbaWiaymZ4. Pour information, les originaux en .xml sont également disponibles, ainsi que le script de conversion `xml_book_parser`. 

> De cette manière, s'il y a des changements à opérer dans la conversion (et le traitement des espaces, notamment), vous pourrez toujours refaire l'opération.





b) Le code présent dans le colab  [Spacy_pipeline](https://colab.research.google.com/drive/1knexo5mQqH9bLstERYefwNx0JtV45qt0) sert à identifier les personnages d'un roman et toutes leurs occurrences à l'aide de Spacy. 
*   La fonction `spacy_char_pipe` prend en input un roman en .txt où chaque ligne est un paragraphe, et enregistre un fichier json contenant la liste de tous les personnages triée par nombre d'occurrences, et la liste des paragraphes contenant ces occurrences.
*   La fonction `spacy_df_pipe` prend en input un roman en .txt où chaque ligne est un paragraphe, et enregistre un fichier .csv contenant un dataframe à cinq colonnes (name, start_pos, end_pos, tag, score) où chaque ligne est une occurrence d'entité nommée reconnue par Spacy. 

Documentez chacune des fonctions présentes dans le colab (à la manière de la documentation de DHTK, par exemple ici https://gitlab.com/dhtk/dhtk/-/blob/testing/dhtk/common/book.py) pour vous assurer d'avoir bien compris le rôle de chacune d'entre elles.

c) A l'aide de la librairie Python `dataclasses` (https://docs.python.org/3/library/dataclasses.html), transformez le code fonctionnel présent dans le colab "Spacy_pipeline" en un code orienté objet. 




> Vous pouvez trouver des tutoriels en ligne pour comprendre les data classes, par exemple ici : http://zetcode.com/python/dataclass/.

In [2]:
!pip install spacy

In [3]:
!python -m spacy download en_core_web_lg  #download the spacy model

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [1]:
import spacy
from spacy import displacy
from spacy.symbols import nsubj, VERB
import numpy as np
from tqdm import tqdm
from collections import Counter, defaultdict
import json
import pandas
from dataclasses import dataclass
import en_core_web_lg

In [2]:
import tkinter as tk
from tkinter import filedialog
# c'est une librairie qui permet de demander un fichier a l'utilisateur

In [3]:
root=tk.Tk()
root.withdraw()
# je sais pas trop ce que c est mais c'est necessaire pour demander le fichier

''

In [5]:
book_path = filedialog.askopenfilename()
book_path
# la j'utilise le tkinter, donc il va prompt une fenetre qui demandera quel fichier tu veux et il retourne le chemin

'/Users/jasonola/Desktop/spacy/textes/BerlinandSansSouciorFredericktheGreatandhisfriends4205.txt'

In [6]:
with open(book_path,"r") as f:
    book = f.read()

In [7]:
nlp = spacy.load("en_core_web_lg")  # load the model

In [8]:
#Insérer votre code ici
# Classe principale Book
@dataclass
class Book:
  path:str
  def get_name(self):
    """Getting the name of the book between the '/'and the '.'"""
    name = self.path.split('/')[-1]
    name = name.split('.')[0]
    return name
  def get_lines(self):
    with open(self.path, 'r', encoding='utf-8', newline='\n') as file:
        content = file.read()
        lines = content.split('\n')
        return lines
  def spacy_ner(self):
    paragraphs = []
    len_diffs = []
    for line in tqdm(self.get_lines()):
        line_length = len(line)
        paragraph = Line.spacy_tag(line) 
        len_diff = line_length - len(paragraph.text)
        paragraphs.append(paragraph)
        len_diffs.append(len_diff)    
    return paragraphs, len_diffs
  def spacy_char_pipe(self):
    name = self.get_name()
    
    paragraphs, _ = self.spacy_ner()
    test = Character(paragraphs)
    characters_sorted, context = test.spacy_characters(paragraphs)
    json_char = {'counter': characters_sorted, 'context': str(context)}
        
    with open(f'./Data/{name}_characters_spacy.json', 'w', encoding='utf-8') as file:
        json.dump(json_char, file)
                  
    return characters_sorted, context
  def spacy_df_pipe(self):
    entities = []
    offset = 0
    
    paragraphs, len_diffs = self.spacy_ner()
    
    for i in range(len(paragraphs)):
        paragraph = paragraphs[i]
        len_diff = len_diffs[i]
        for ent in paragraph.ents:

            entity = {
                'name': ent.text, 
                'start_pos': offset + ent.start_char, 
                'end_pos': offset + ent.end_char, 
                'tag': ent.label_, 
                'score': 1, 
            }

            entities.append(entity)

        offset += len(paragraph.text) + len_diff + 1

    entity_df = pandas.DataFrame(entities)

    name = self.get_name()
    entity_df.to_csv(f'./Data/{name}_spacy.csv')
    
    return entity_df



In [9]:
# spacy_tag dans une autre classe qu'on appele depuis spacy_ner dans la classe Book !
@dataclass
class Line:
  def spacy_tag(self):
    paragraph = nlp(self)
    return paragraph


In [10]:
#Classe Character
@dataclass
# Determine if the entity is a person
class Character:

  def __init__(self, paragraphs):
    self.paragraphs = paragraphs

  def is_char(self, entity):
    return entity.label == 'PERSON'

  def count_char(self, paragraph):
    nb_char_par = Counter()

    for entity in paragraph.ents:
      if self.is_char(entity):
        nb_char_par[entity.text] += 1

    return nb_char_par

  def sort_char(self, char_list):
    characters_sorted = char_list.most_common()
    return characters_sorted

  def spacy_characters(self, paragraphs):
    nb_characters = Counter()
    context = defaultdict(list)

    for paragraph in paragraphs:
      nb_char_par = self.count_char(paragraph)
      nb_characters.update(nb_char_par)
      for entity in nb_char_par:
        context[entity].append(paragraph)

    characters_sorted = self.sort_char(nb_characters)
    return characters_sorted, context

In [12]:
# Tests of methods, je fais chaque methode l'une apres l'autre il y a des methodes 
# qui demandent d'autres methodes genre spacy_ner qui demande spacy_tag, enlevez juste le # devant un et run
p = Book(book_path)
#p.get_name() # Works !
#p.get_lines() # Works !
#p.spacy_ner() # Works !!!


#paragraphs, _ = p.spacy_ner()
#test = Character(paragraphs)
#test.spacy_characters()
#p.spacy_char_pipe() # ca marche mais ca retourne une liste vide
#p.spacy_df_pipe() # ca marche quand je commente la partie .to_csv

['AUTHOR OF JOSEPH II. AND HIS COURT, FREDERICK THE GREAT AND HIS COURT, MERCHANT OF BERLIN, ETC. TRANSLATED FROM THE GERMAN BY MRS. CHAPMAN COLEMAN AND HER DAUGHTERS',
 'I. The Two Sisters II. The Tempter III. The Wedding-Festival of the Princess Ulrica IV. Behind the Curtain V. A Shame-faced King VI. The First Rendezvous VII. On The Balcony VIII. The First Cloud IX. The Council of War X. The Cloister of Camens XI. The King and the Abbot XII. The Unknown Abbot XIII. The Levee of a Dancer XIV. The Studio XV. The Confession XVI. The Traitor XVII. The Silver-Ware XVIII. The First Flash of Lightning',
 "I. The Actors in Halle II. The Student Lupinus III. The Disturbance in the Theatre IV. The Friends V. The Order of the King VI. The Battle of Sohr VII. After the Battle VIII. A Letter Pregnant with Fate IX. The Return to Berlin X. Job 's Post XI. The Undeceived XII. Trenck 's First Flight XIII. The Flight XIV. “I will” XV. The Last Struggle for Power XVI. The Disturbance in the Theatre XVI

In [12]:
#Insérer votre code ici

exemple_string = nlp("This text serves as a very nice exemple. And this part of the text too")


# Code pour utiliser un texte importé
#chemin = "/content/AHorsesTalebyMarkTwain1086.txt"
#with open(chemin, "r") as f:
  #my_book = f.read()
  #text_exemple = nlp(my_book)

# Ces deux variables permettent de mettre du texte en gras
BOLD = '\033[1m'
END = '\033[0m'

@dataclass
class SpacyFeatures:
  # Affichage des POS tag
  def get_tag(self):
    for token in self:
      print(BOLD + "Text  :  Type  :  children" + END)
      print(f"{token.text} | {token.pos_} | {[child for child in token.children]} \n")
  # Affichage des noun chunks et des dépendences 
  def noun_chunk(self):
    for chunk in self.noun_chunks:
      print(BOLD + "Text  :  Main Noun  :  Dependency  :  Completed object" + END)
      print(f"{chunk.text} | {chunk.root.text} | {chunk.root.dep_} | {chunk.root.head.text} \n")
  #Affichage de verbes
  def get_verbs(self):
    verbs = set()
    for possible_subject in self:
      if possible_subject.dep == nsubj and possible_subject.head.pos == VERB:
        verbs.add(possible_subject.head)
    print(verbs)

  #Affichage du parsing tree
  def parsing_tree(self):
    displacy.render(self, style='dep', jupyter=True, options={'distance': 130})
    displacy.render(self, style='ent', jupyter=True)
  
  #Séparateur de phrases
  def get_sentence(self):
    for sent in self.sents:
      print(sent.text)

  #Trouver des entités nommées, des phrases et des concepts
  def get_entity(self):
    for entity in self.ents:
      print(entity.text, entity.label_)

SpacyFeatures.noun_chunk(exemple_string)
#SpacyFeatures.parsing_tree(exemple_string)


Text  :  Main Noun  :  Dependency  :  Completed object
This text | text | nsubj | serves 

Text  :  Main Noun  :  Dependency  :  Completed object
a very nice exemple | exemple | pobj | as 

Text  :  Main Noun  :  Dependency  :  Completed object
And this part | part | ROOT | part 

Text  :  Main Noun  :  Dependency  :  Completed object
the text | text | pobj | of 



d) A l'aide de la librairie Spacy https://spacy.io/, intégrez les autres fonctions disponible dans Spacy  (comme montré [ici](https://spacy.io/usage/linguistic-features))  dans la classe que vous avez créée au point *b)*


> Pour installer Spacy : suivre les consignes présentes sur ce lien https://spacy.io/usage.



